## Big Data Engineering Semester Project

Before we begin importing our dataset and setting up Kafka Producer, we need to make sure we have a zookeper service running. When working with Apache Kafka, ZooKeeper is primarily used to track the status of nodes in the Kafka cluster and maintain a list of Kafka topics and messages.

In our case, we are using an image of zookeper that is running on a Docker container. To run the container, simply navigate to the folder containig docker-compose file of zookeper and execute command 'docker-compose up -d' to start the process.

Another tool that was used in our project is Offset Explorer, an open-source application that allows to connedct to Kafka Cluster and visually see all the topics and messages. Offset Exploer can dowloaded via this link - https://www.kafkatool.com/download.html

#### Setting up Kafka Producer to write Chicago Crime data to topic

Chicago crime data can be accessed via Socrata API call to the Chicago Police department, which website can be found under this link.
https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2

Now creating an instance of Spark Producer and bootstraping it to the local server where Kafka is running.

The next step is to gather the Crime data we need using the Socraty API interface, limiting the qury output to 100.000 lines to increase performance.

Finally, loop over the output and send each data point to the Kafka Topic.

In [15]:

import time
import json
from json import dumps
from kafka import KafkaProducer
from time import sleep
import requests as req

# Create producer
producer = KafkaProducer(
    bootstrap_servers='localhost:29092', #Kafka server
    value_serializer=lambda v: json.dumps(v).encode('utf-8') #json serializer
    )

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)


# change the timeout variable to an arbitrarily large number of seconds to avoid connection error
client.timeout = 120

results = client.get("ijzp-q8t2", limit=100000)

# Convert to pandas DataFrame if needed
#results_df = pd.DataFrame.from_records(results)

for i in results:
    producer.send('crimeData-chicago_100k', i)
    #print("Getting new data...")





ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:29092 <connected> [IPv4 ('127.0.0.1', 29092)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:29092 <connected> [IPv6 ('::1', 29092, 0, 0)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:29092 <connected> [IPv4 ('127.0.0.1', 29092)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:29092 <connected> [IPv6 ('::1', 29092, 0, 0)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:29092 <connected> [IPv4 ('127.0.0.1', 29092)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:29092 <connected> [IPv6 ('::1', 29092, 0, 0)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:29092 <connected> [IPv4 ('127.0.0.1', 29092)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:29092 <connected> [IPv6 ('::1', 29092, 0, 0)

In [ ]:
producer.close()

Double check that we got the correct data.

In [6]:
for i in results:
    print(i)

{'id': '12736693', 'case_number': 'JF287903', 'date': '2022-06-20T23:56:00.000', 'block': '007XX W GARFIELD BLVD', 'iucr': '0486', 'primary_type': 'BATTERY', 'description': 'DOMESTIC BATTERY SIMPLE', 'location_description': 'APARTMENT', 'arrest': False, 'domestic': True, 'beat': '0711', 'district': '007', 'ward': '20', 'community_area': '68', 'fbi_code': '08B', 'x_coordinate': '1172439', 'y_coordinate': '1868272', 'year': '2022', 'updated_on': '2022-06-27T16:49:20.000', 'latitude': '41.793979431', 'longitude': '-87.643208858', 'location': {'latitude': '41.793979431', 'longitude': '-87.643208858', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}, ':@computed_region_awaf_s7ux': '12', ':@computed_region_6mkv_f3dw': '21559', ':@computed_region_vrxf_vc4k': '66', ':@computed_region_bdys_3d7i': '113', ':@computed_region_43wa_7qmu': '4', ':@computed_region_rpca_8um6': '11', ':@computed_region_d9mm_jgwp': '17', ':@computed_region_d3ds_rm58': '135'}
{'id': '12736637', 'cas